In [16]:
print("ok")

ok


In [17]:
%pwd

'c:\\MEDICAL\\medical-chatbot-AI\\research'

In [18]:
import os
os.chdir("../")

In [19]:
%pwd

'c:\\MEDICAL\\medical-chatbot-AI'

In [20]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [22]:
extracted_data = load_pdf_file(data='C:\medical\Data')

In [23]:
#extracted_data

In [24]:
# Split the Data into Text Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter 

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [25]:
text_chunks = text_split(extracted_data)

print("Length of Text Chunks:", len(text_chunks))

Length of Text Chunks: 5859


In [26]:
from langchain.embeddings import HuggingFaceEmbeddings


In [27]:
# 1. Define the function (you already did this)
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [28]:
pip install --upgrade sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [29]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


In [30]:
embeddings = download_hugging_face_embeddings()


C:\Users\kisho\AppData\Local\Temp\ipykernel_10968\1038990354.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [31]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [32]:
#query_result

In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
PINECONE_API_KEY=os.environ.get('pcsk_4dRr3v_PyJzmcQsgBWkSb9AL9c3rMb1QDVTBy128Mbo1uQqezdCXKER2WtwsGy3iSHNNni')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [35]:
!pip install google protobuf
!pip install sentence-transformers
!pip install sentence-transformers
!pip install -U langchain-community
!pip install --upgrade protobuf==6.31.0
!pip install pinecone
!pip install langchain_pinecone


In [37]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec




# Initialize Pinecone client with API key
pc = Pinecone(api_key="pcsk_4dRr3v_PyJzmcQsgBWkSb9AL9c3rMb1QDVTBy128Mbo1uQqezdCXKER2WtwsGy3iSHNNni")

# Define index name and configuration
index_name = "medibot-v2"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,  # must match your embedding size
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medibot-v2",
    "metric": "cosine",
    "host": "medibot-v2-v1djjgq.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [38]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_4dRr3v_PyJzmcQsgBWkSb9AL9c3rMb1QDVTBy128Mbo1uQqezdCXKER2WtwsGy3iSHNNni"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [40]:
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [41]:
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    
    index_name=index_name,
    embedding=embeddings)

In [42]:
docsearch

In [43]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # fetch top 3 similar documents
)

# Invoke the retriever with a query
retrieved_docs = retriever.invoke("What is Acne?")


In [44]:
retrieved_docs

[Document(id='67593c73-58d1-4441-b54a-2bd7a1877c67', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\medical\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='8ab74b09-d96d-40aa-8245-b8f650b9f91d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\medical\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='e713a285-4c1b-4a63-94d2-b4585dde1dd3', metadata={'creationdate': '2004-12-18T17:00

In [45]:
#brief record from book

In [46]:
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1}:\n")
    print(doc.page_content)


Document 1:

GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26

Document 2:

GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26

Document 3:

GALE ENCYCLOPEDIA OF MEDICINE 2 25
Acne
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceous
glands become inflamed. (Photograph by Biophoto Associ-
ates, Photo Researchers, Inc. Reproduced by permission.)
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25


In [47]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.4,
    max_tokens=300,
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",  # or any supported Together model
    openai_api_key="f99e4e137a324ca884a38408a1991c00c7ec67fbe53b5fce65ddb317e71c5a8b",
    openai_api_base="https://api.together.xyz/v1"
)


In [48]:
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
  # or any other chat model you are using

# Define the system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\nContext:\n{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [49]:
!pip install -U langchain-openai



In [50]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)


In [51]:
response = llm.invoke("What is acne?")
print(response.content)


 Acne is a common skin condition that occurs when the hair follicles become plugged with oil and dead skin cells. It can cause whiteheads, blackheads, or pimples to appear on the skin, most commonly on the face, forehead, chest, upper back and shoulders. Acne can affect people of all ages, but it is particularly common among teenagers.

Acne occurs when the sebaceous glands, which are small glands found near the surface of the skin, produce an excess amount of sebum (oil). This oil mixes with dead skin cells and can clog the pores, leading to the development of acne.

There are several factors that can contribute to the development of acne, including:

* Hormonal changes: Hormonal fluctuations during puberty, menstruation, pregnancy, or due to conditions such as polycystic ovary syndrome (PCOS) can cause the sebaceous glands to produce more sebum.
* Bacteria: The presence of certain bacteria on the skin can contribute to the development of acne.
* Medications: Certain medications, such

In [52]:
response = llm.invoke("who is virat kholi?")
print(response.content)


 Virat Kohli is an Indian international cricketer who is currently the captain of the Indian national team in all formats of the game. He is widely regarded as one of the best batsmen in the world and is known for his aggressive and dynamic playing style.

Kohli was born on November 5, 1988, in Delhi, India. He made his debut for the Indian national team in 2008 and has since established himself as one of the team's key players. He has scored over 22,000 runs in international cricket, including more than 7,000 runs in Test matches and over 11,000 runs in One Day Internationals (ODIs).

Kohli is known for his ability to score quickly and consistently, and he has a wide range of shots in his repertoire. He is particularly strong on the off side and is known for his ability to find the boundary with regularity.

In addition to his success on the field, Kohli is also a popular and influential figure off the field. He is the captain of the Indian Premier League (IPL) team Royal Challengers 